<a href="https://colab.research.google.com/github/jooeun921/Big-Data-Analyst/blob/main/Part06_Section_02_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Section 02 학습 : 로지스틱 회귀 분석

#### 오즈 / 오즈비

In [ ]:
# 확률의 오즈(odds)란 어떤 사건이 발생할 확률과 그 사건이 발생하지 않을 확률의 비율.

import pandas as pd
import numpy as np

admission_data = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/admission.csv')
print(admission_data.shape)
print(admission_data.head( ))

In [ ]:
# admit(입학 허가 여부)에 대한 오즈비 계산하기

p_hat = admission_data['admit'].mean()
print(np.round(p_hat / (1 - p_hat), 3))

In [ ]:
# 범주형 변수를 사용한 오즈 계산

unique_ranks = sorted(admission_data['rank'].unique())
print(*unique_ranks)

In [ ]:
# print(admission_data.groupby.__doc__)

grouped_data = admission_data.groupby(by = 'rank')[['admit']].mean()
grouped_data['odds'] = grouped_data['admit'] / (1 - grouped_data['admit'])
grouped_data = grouped_data.rename(columns = {'admit' : 'p_admit'})
print(grouped_data)

# print(grouped_data.rename.__doc__)

In [ ]:
# 오즈비를 활용하여 확률 역산하기
# p = odds / (1 + odds)

print(np.round(1.178 / (1.178 + 1), 3))

In [ ]:
# 로그 오즈
# 로지스틱 회귀에서는 확률을 직접 독립변수들의 선형 결합으로 모델링할 수 없다.
# 확률은 0-1 사이의 값을 가지지만, 모델링하는 선형결합은 마이너스 무한대에서 무한대 사이의 값을 가지기 때문.
# 따라서 오즈에 로그를 씌워 사용함으로서, 독립변수들의 선형 결합으로 모델링할 수 있게 됨.

import numpy as np
import matplotlib.pyplot as plt

p = np.arange(0, 1.01, 0.01)
log_odds = np.log(p / (1-p))

plt.plot(p, log_odds)
plt.xlabel('p')
plt.ylabel('log_odds')
plt.title('Plot of log odds')
plt.show()

In [ ]:
# 로지스틱 회귀계수 예측과 분석

odds_data = admission_data.groupby('rank').agg(p_admit = ('admit', 'mean')).reset_index()
odds_data['odds'] = odds_data['p_admit'] / (1 - odds_data['p_admit'])
odds_data['log_odds'] = np.log(odds_data['odds'])
print(odds_data)

In [ ]:
from statsmodels.formula.api import logit, ols

model_ols = ols('log_odds ~ rank', data = odds_data).fit()
print(model_ols.summary())

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(odds_data['rank'], odds_data['log_odds'], label = 'Data Points')

x = odds_data['rank']
y = odds_data['log_odds']
coefficients = np.polyfit(x, y, 1)
poly_eq = np.poly1d(coefficients)

plt.plot(x, poly_eq(x), color = 'r', label = 'Regression Line')

plt.xlabel('Rank')
plt.ylabel('Log Odds')
plt.title('Scatter plot with regression line')
plt.legend()
plt.show()

In [ ]:
selected_data = odds_data[['rank', 'p_admit', 'odds']]
selected_data['odds_frac'] = selected_data['odds'] / selected_data['odds'].shift(1, fill_value = selected_data['odds'].iloc[0])
print(selected_data)

In [ ]:
# 오즈를 이용한 확률 역산하기

rank_vec = np.array([1, 2, 3, 4])
result = np.exp(0.6327 - 0.5675 * rank_vec) / (1 + np.exp(0.6327 - 0.5675 * rank_vec))

print(result)

#### 로지스틱 회귀 분석

In [ ]:
admission_data = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/admission.csv')

admission_data['rank'] = admission_data['rank'].astype('category')
admission_data['gender'] = admission_data['gender'].astype('category')

In [ ]:
from statsmodels.formula.api import logit, glm
import statsmodels.api as sm

model_logit = logit('admit ~ gre + gpa + rank + gender', data = admission_data).fit()
print(model_logit.summary())


In [ ]:
model_glm = glm('admit ~ gre + gpa + rank + gender', data = admission_data, family = sm.families.Binomial()).fit()
print(model_glm.summary())

In [ ]:
# 각 계수 검정하기.

from scipy.stats import norm

result1 = 0.002256 / 0.001094
result2 = 2 * (1 - norm.cdf(result1))

print(result1)
print(result2)

In [ ]:
model_logit.pvalues

In [ ]:
# 각 Odds ratio에 대한 신뢰구간 구하기
# model.conf_int()는 model에서 계산한 5%에 대한 신뢰구간 값을 출력함.

odds_ratios = pd.DataFrame({
    'OR' : model_logit.params,
    'Lower CI' : model_logit.conf_int()[0],
    'Upper CI': model_logit.conf_int()[1],
})

odds_ratios = np.exp(odds_ratios)
print(odds_ratios)

In [ ]:
print(model_logit.conf_int(alpha = 0.10))

In [ ]:
model_logit.params

In [ ]:
# 신뢰구간 직접 계산하기
# model_logit.params[5] = 0.002256
# model_logiti.bse => 표준오차, 0.001094
from scipy.stats import norm

a = round(model_logit.params.iloc[5] - norm.ppf(0.975) * 0.001094, 3)
b = round(model_logit.params.iloc[5] + norm.ppf(0.975) * 0.001094, 3)

glue_str = f"({a}, {b})"
print(glue_str)

In [ ]:
# 오즈비에 대한 신뢰구간

a = round(np.exp(a), 3)
b = round(np.exp(b), 3)

glue_str = f"({a}, {b})"
print(glue_str)

In [ ]:
model_logit.conf_int()

In [ ]:
model_logit.summary()

In [ ]:
# 모델의 유의성을 체크하기 위한 우도비(likelihood) 검정통계량 계산하기

test_statistic = np.round(-2 * (model_logit.llnull - model_logit.llf), 3)
print(f"Test statistic : {test_statistic}")

In [ ]:
# 우도비에 대한 모형 전체 검정 p-value

from scipy.stats import chi2

df = model_logit.df_model - 0

p_value = chi2.sf(test_statistic, df)
print("p-value", np.round(p_value, 10))

In [ ]:
# deviance
print("deviance = ", np.round(-2 * model_logit.llf, 3))
print("null deviance = ", np.round(-2 * model_logit.llnull, 3))

In [ ]:
# 모델 자유도
model_logit.df_model

In [ ]:
# 예측
from sklearn.metrics import roc_auc_score

new_data = pd.DataFrame({
 'gre': [400, 700, 750, 500], # 새로운 GRE 점수
 'gpa': [3.5, 3.8, 3.9, 3.2], # 새로운 GPA 점수
 'rank': [2, 1, 4, 3], # 새로운 대학 순위
 'gender': ['M', 'F', 'F', 'M'] # 새로운 지원자의 성별
})

y_true = pd.Series([0, 1, 0, 0])

new_data['admit_prob'] = model_logit.predict(new_data)

auc_score = roc_auc_score(y_true, new_data['admit_prob'])

print(new_data[['gre', 'gpa', 'rank', 'gender', 'admit_prob']])
print('AUC score :', auc_score)

### Section 02 연습문제 : 로지스틱 회귀 분석
```
odds_ratio = np.exp(model.params)
```

In [ ]:
#1-3 다음 데이터는 몸무게와 키, 나이, 그리고 수입에 대한 정보를 담고 있다. 데이터를 사용하여 다음의 물음에 답하시오.

import pandas as pd
import numpy as np

# 예제 데이터 생성
np.random.seed(42)
n_samples = 210
X = np.random.randn(n_samples, 4)
y = (X[:, 0] + X[:, 1] * 0.5 + np.random.randn(n_samples) * 0.5 > 0).astype(int)
df = pd.DataFrame(X, columns=['weight', 'height', 'age', 'income'])
df['gender'] = y

# 데이터 확인
print(df.head())

In [ ]:
#1 성별 변수(gender)를 사용하여 몸무게 변수(weight)에 대한 로지스틱 회귀모델을 적합하고, 해당하는 오즈비(weight)를 계산하시오.

from statsmodels.formula.api import logit

model1 = logit("gender ~ weight", data = df).fit()

model1.summary()

In [ ]:
p_weight = model1.params
odds_ratio = np.exp(p_weight)

print(odds_ratio)

In [ ]:
#2 성별 변수를 주어진 4개 변수를 사용하여 로지스틱 회귀모델을 적합했을 때, residual deviance를 계산하시오.

model2 = logit("gender ~ weight + height + age + income", data = df).fit()

print(round(model2.llf * -2, 3))

In [ ]:
#3 1번 문제의 모델(몸무게를 독립변수로 사용) 데이터를 학습 데이터와 평가 데이터(90개로 설정)로 분류한 후, 오분류율을 계산하시오. (소수점 넷째 자리에서 반올림)

from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size = 90, random_state = 42)

from sklearn.metrics import accuracy_score

y_true = df_test['gender']

model3 = logit("gender ~ weight", data = df_train).fit()
y_pred = model3.predict(df_test[['weight']]) > 0.5

error_rate = 1 - accuracy_score(y_true, y_pred)

print(np.round(error_rate, 3))

In [ ]:
#4-6 다음 당뇨병 데이터는 체질량지수(bmi), 평균 혈압(bp), 혈청(s), 진행 정도(target) 등에 대한 정보를 담고 있다. 데이터를 사용하여 다음의 물음에 답하시오.

import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes

diabetes = load_diabetes(as_frame = True)
df = diabetes.frame
print(df.head( ))

In [ ]:
#4 target 변수를 중앙값을 기준으로 낮으면 0, 높으면 1로 이진화한 후, 로지스틱 회귀모델을 적합시키고, 통계적으로 유의하지 않은 변수의 개수를 구하시오.
# (조건 : 유의 수준은 0.05로 설정, 상수항 계수가 유의할 경우 변수 개수에 포함. s1~s6 변수 제거)

target_median = df['target'].median()
df['target'] = (df['target'] >= target_median).astype(int)

from statsmodels.formula.api import logit

model4 = logit('target ~ age + sex + bmi + bp', data = df).fit()

print(model4.summary())

print("통계적으로 유의하지 않은 변수 : age, sex intercept")
print((model4.pvalues > 0.05).sum())

In [ ]:
#5 4번 문제에서 유의한 변수들만 사용하여 다시 로지스틱 회귀 적합하고, 유의한 변수들의 회귀계수 평균을 구하시오.

model5 = logit('target ~ bmi + bp', data = df).fit()
print(model5.params.mean())

In [ ]:
#6 4번 문제에서 나이가 1 단위 증가할 때 오즈비를 계산하시오.

age_param = model4.params.iloc[1]
# model4.params

odds_ratio = np.exp(age_param * 1)

print(odds_ratio)